# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [4]:
# URL para a API
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'

# Requisição GET para a API
response = requests.get(url)

# Verificar o status da requisição
if response.status_code == 200:
    # Carregar os dados em formato JSON
    data = response.json()
    # Transformar os dados em um DataFrame
    df = pd.DataFrame(data['items'])
    
    # Mostrar as primeiras linhas do DataFrame para entender sua estrutura
    print(df.head())
    
    # Identificar os três status mais frequentes e suas frequências
    status_frequencies = df['status'].value_counts().head(3)
    print("Três status mais frequentes e suas frequências:")
    print(status_frequencies)
    
    # Construir uma nova variável com o ano do status
    df['data_status'] = df['data_status'].str.slice(0, 4)
    
    # Indicar a frequência de cada ano
    year_frequencies = df['data_status'].value_counts()
    print("Frequência de cada ano:")
    print(year_frequencies)
else:
    print(f"Erro na requisição: {response.status_code}")


   id_pleito tipo_interessado         interessado  cod_ibge  uf  \
0      68563        Município         Nova Iguaçu   3303500  RJ   
1      65407        Município            Itaboraí   3301900  RJ   
2      34099        Município             Resende   3304201  RJ   
3      61653        Município         Teresópolis   3305802  RJ   
4      11707        Município  São João de Meriti   3305109  RJ   

                num_pvl                                   status  \
0  PVL02.000551/2024-39  Assinado pelo interessado (retificação)   
1  PVL02.001946/2023-78                       Arquivado a pedido   
2  PVL02.003090/2019-99                        Deferido (PVL-IF)   
3  PVL02.007127/2022-53                                 Deferido   
4                  None                                 Deferido   

           num_processo        data_protocolo  \
0  17944.002171/2024-00  2024-05-28T20:07:11Z   
1  17944.102785/2023-00  2023-06-30T18:15:09Z   
2                  None                  

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [6]:
def consulta_pvl(uf, tipo_interessado):
    url = f'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['items'])
        return df
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None

# Consultando os dados para MG e filtrando pelo status 'Arquivado por decurso de prazo'
df_mg = consulta_pvl('MG', 'Estado')
mg_arquivado = df_mg[df_mg['status'] == 'Arquivado por decurso de prazo']
quantidade_mg_arquivado = mg_arquivado.shape[0]
print(f"Quantidade de solicitações em MG com status 'Arquivado por decurso de prazo': {quantidade_mg_arquivado}")

# Consultando os dados para municípios da BA e filtrando pelo status 'Deferido'
df_ba_municipios = consulta_pvl('BA', 'Município')
ba_deferido = df_ba_municipios[df_ba_municipios['status'] == 'Deferido']
municipio_mais_deferido = ba_deferido['interessado'].value_counts().idxmax()
print(f"Município da BA com mais solicitações deferidas: {municipio_mais_deferido}")

# Consultando os dados para o Estado da BA e salvando em um arquivo .csv
df_ba_estado = consulta_pvl('BA', 'Estado')
df_ba_estado.to_csv('solicitacoes_ba_estado.csv', index=False)
print("Arquivo 'solicitacoes_ba_estado.csv' salvo com sucesso.")


Quantidade de solicitações em MG com status 'Arquivado por decurso de prazo': 1
Município da BA com mais solicitações deferidas: Luís Eduardo Magalhães
Arquivo 'solicitacoes_ba_estado.csv' salvo com sucesso.
